# Why HTMap?

## Setup

This tutorial assume that you've already installed HTMap.
See [the installation instructions](/faq.html#how-do-i-install-htmap) for details!

This tutorial assumes that you're working in a Jupyter Notebook.
It will work just as well in the Python REPL.
Later, once you get a hang of the way that maps work, you'll be ready to use HTMap in scripts as well.
Either way, you'll need to be on a computer that can see an HTCondor pool!

This tutorial assumes that you have already set up your dependency management, as described in [Dependency Management](/dependencies.html).
If your HTCondor pool supports Docker, you'll be good to go with the default settings.

## The Problem

Suppose you've been given the task of writing a function that doubles numbers.
This is hardly a challenge:

In [1]:
def double(x):
    return 2 * x

If you want to double a list of numbers, you might do something like

In [2]:
doubled = [double(x) for x in range(10)]
print(doubled)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


or we can use the built-in function `map()`, which applies a function to each element of an iterable (like a list):

In [3]:
mapped = map(double, range(10))
print(mapped)
doubled = list(mapped)
print(doubled)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


In both cases, `doubled` is the list `[0, 2, 4, ...]`.
The reason we need the `list` call is that `map` actually returns an *iterator* over the results, not the results themselves.
So you need to iterate over it to get the output, which is what `list` does: iterate over its input and put the elements in a list.

Now suppose that, for some reason, you want to double *a lot* of numbers.
So many numbers that you can't bear to do all the work on your own computer.
It takes days to multiply all the numbers, and if your program crashes halfway through, you lose all of of your progress and have to start over.
You're losing sleep, and your boss is breathing down your neck because they need those numbers doubled *now*.

Luckily, you remember that you have access to an HTCondor high-throughput computing pool.
Since each of your function calls is isolated from all the others, the computers in the pool don't need to talk to each other at all, and you can achieve a huge speedup.
The pool can run your code on hundreds or thousands of computers simultaneously, storing the inputs and outputs for you and recovering from individual errors gracefully.
It's the perfect solution.

The problem is: *how do you get your code running in the pool?*

## The Solution

With HTMap, it's like this:

In [4]:
import htmap

mapped = htmap.transient_map(double, range(10))
print(mapped)
doubled = list(mapped)
print(doubled)

<TransientMap(map = <Map(map_id = tmp-1541354990-0)>)>
[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


It may take some time for the second `print` to run.
During that time, the individual **components** of your **map** are being run out on the cluster.
Once they all finish, you'll get the list of numbers back.
As you can see, the output is identical to what you would get from running the function locally.